## 0. Imports and Helper Functions

In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models  
from tensorflow.keras.preprocessing.image import ImageDataGenerator  
from tensorflow.keras.utils import to_categorical  
import pandas as pd   
import matplotlib.pyplot as plt  
import numpy as np
from PIL import Image 

## 1. Load and prepare the CIFAR10 dataset

In [32]:
class_names =  ['AIRPLANE', 'AUTOMOBILE', 'BIRD', 'CAT', 'DEER', 'DOG', 'FROG', 'HORSE', 'SHIP', 'TRUCK']

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train / 255.0
y_train = to_categorical(y_train, len(class_names))

x_test = x_test / 255.0
y_test = to_categorical(y_test, len(class_names))

## 2. Define the Model
We are going to create the convolutional base

In [33]:
def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPool2D((2,2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',))
    model.add(layers.MaxPool2D((2,2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same',))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same',))
    model.add(layers.MaxPool2D((2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

## 3. Initial Training with Data Augmentation

In [34]:
datagen = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=3/32,
    height_shift_range=3/32
)
    
initial_model = create_model()

history = initial_model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=25,
    validation_data=(x_test, y_test))

Epoch 1/25
782/782 [==============================] - 22s 26ms/step - loss: 1.6416 - accuracy: 0.3926 - val_loss: 1.2240 - val_accuracy: 0.5595
Epoch 2/25
782/782 [==============================] - 20s 25ms/step - loss: 1.1901 - accuracy: 0.5740 - val_loss: 1.0507 - val_accuracy: 0.6331
Epoch 3/25
782/782 [==============================] - 20s 25ms/step - loss: 1.0048 - accuracy: 0.6436 - val_loss: 0.9448 - val_accuracy: 0.6696
Epoch 4/25
782/782 [==============================] - 20s 25ms/step - loss: 0.8680 - accuracy: 0.6949 - val_loss: 0.8024 - val_accuracy: 0.7183
Epoch 5/25
782/782 [==============================] - 20s 25ms/step - loss: 0.7787 - accuracy: 0.7268 - val_loss: 0.7433 - val_accuracy: 0.7422
Epoch 6/25
782/782 [==============================] - 19s 25ms/step - loss: 0.7155 - accuracy: 0.7504 - val_loss: 0.7216 - val_accuracy: 0.7474
Epoch 7/25
782/782 [==============================] - 20s 26ms/step - loss: 0.6695 - accuracy: 0.7678 - val_loss: 0.6712 - val_accuracy:

In [35]:
df = pd.DataFrame.from_dict(history.history)
df["N_Epochs"] = range(1,len(df)+1)
df

,loss,accuracy,val_loss,val_accuracy,N_Epochs
0,1.641556,0.39260,1.223987,0.5595,1
1,1.190053,0.57402,1.050682,0.6331,2
2,1.004844,0.64358,0.944823,0.6696,3
3,0.868039,0.69488,0.802400,0.7183,4
4,0.778743,0.72676,0.743317,0.7422,5
5,0.715535,0.75040,0.721643,0.7474,6
6,0.669476,0.76780,0.671227,0.7721,7
7,0.632030,0.77878,0.646522,0.7796,8
8,0.604708,0.78842,0.622420,0.7849,9
9,0.572183,0.80090,0.654009,0.7854,10


## 4. Visualization of Training & Validation Accuracy 

In [36]:
from taipy import Gui 

Gui(page="<|{df}|chart|x=N_Epochs|y[1]=accuracy|y[2]=val_accuracy|>").run(port=5000)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


## 5. Building a complete application with Taipy GUI

In [11]:
port = 5001

c:\Users\alber\anaconda3\envs\image_classification_demo\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


##### Defining state variables

In [24]:
model  = initial_model

# Parameters for models & training
epochs = 1
input_model_name = "model" 

# Parameters for trained model
trained_model_path = ""

# Parameters for CIFAR dataset
cifar_image_index  = 10
cifar_image_path = "images/sample/taipy.jpg"
cifar_predicted_label = 'NA'
cifar_true_label = 'NA'

# Parameters for online image
online_image_url  = "URL"
online_image_path = "images/sample/airplane.jpg" 
online_image_count = 0
online_image_predicted_label  = 'NA' # predicted label for the online image

##### Taipy GUI

In [25]:
#P1
from taipy.gui import invoke_long_callback, notify

p1 = """
<center><h1>Image Classification CNN</h1></center>

<|layout|columns=1 3|
<|
## PARAMETERS
Enter chosen optimal numper of epochs:   
<|{epochs}|input|>  


Register model name:   
<|{input_model_name}|input|>

Train the model with the Training + Validation sets:  
<|START TRAINING|button|on_action=train_button|>  

### Upload Trained Model
<|{trained_model_path}|file_selector|label=Upload trained model|on_action=load_trained_model|extensions=.h5|>
|>

<|
<center><h2> Val_loss and Accuracy </h2></center>
<|{df}|chart|x=N_Epochs|y[1]=accuracy|y[2]=val_accuracy|>
|>
|>
___
"""

def merged_train(model,number_of_epochs,name):
    # merge the training and validation sets
    x_all = np.concatenate((x_train, x_test))
    y_all = np.concatenate((y_train, y_test))

    # train with the merged dataset
    history = model.fit(
        datagen.flow(x_all, y_all, batch_size=64),
        epochs=number_of_epochs)

    model.save("saved_models/{}.h5".format(name),save_format='h5')
    print("TRAINING & SAVING COMPLETED!")

def train_button(state):
    notify(state, "info", "Started training model with {} epochs".format(state.epochs), True, 1000)
    model = create_model()
    invoke_long_callback(state,merged_train,[model, int(state.epochs), state.input_model_name])

def load_trained_model(state):
    loaded_model = tf.keras.models.load_model(state.trained_model_path)
    state.model = loaded_model

In [26]:
import urllib

p2 = """ 
<|layout|columns=1 3|
<|
### CIFAR10 Images Prediction
Enter CIFAR10 image index:  |

<|{cifar_image_index}|input|>  
<|PREDICT CIFAR IMAGE|button|on_action=predict_cifar_image|>

<|{cifar_image_path}|image|height=100px|width=100px|>

##Predicted label: <|{cifar_predicted_label}|>  
##True label: <|{cifar_true_label}|>

|>

<|
###Paste an online image link here for prediction:  

<|{online_image_url}|input|on_action=load_online_image|>  

<center> <|{online_image_path}|image|height=300px|width=300px|> </center>

<|PREDICT ONLINE IMAGE|button|on_action=predict_online_image|>

## Predicted label: <|{online_image_predicted_label }|>
|>
|>
"""

def predict_cifar_image(state):
    #Retrieve the cifar image at the specified index and save as PIL Image obj
    cifar_img_idx = int(state.cifar_image_index )
    cifar_img_data = x_test[cifar_img_idx]
    cifar_img = Image.fromarray(np.uint8(cifar_img_data*255))
    cifar_img.save("images/cifar10_saved/{}.jpg".format(cifar_img_idx))

    #Predict the label of the CIFAR image
    img_for_pred = np.expand_dims(x_test[cifar_img_idx], axis=0)
    cifar_img_pred_label = np.argmax(state.model.predict(img_for_pred))
    cifar_img_true_label = y_test[cifar_img_idx].argmax() 
    
    #Update the GUI
    state.cifar_image_path = "images/cifar10_saved/{}.jpg".format(cifar_img_idx)
    state.cifar_predicted_label = str(class_names[cifar_img_pred_label])
    state.cifar_true_label = str(class_names[cifar_img_true_label])

def load_online_image(state):
    urllib.request.urlretrieve(state.online_image_url, "images/online_image.jpg")
    state.online_image_path = "images/online_image.jpg"

def predict_online_image(state):
    #Retrieve & save online image in order to show on the image box
    urllib.request.urlretrieve(state.online_image_url , "images/saved_images/{}.jpg".format(state.online_image_count))
    state.online_image_path = "images/saved_images/{}.jpg".format(state.online_image_count)

    #Predict the label of the online image
    img_array = tf.keras.utils.load_img(state.online_image_path, target_size=(32, 32))
    image = tf.keras.utils.img_to_array(img_array)  # (height, width, channels)
    image = np.expand_dims(image, axis=0) / 255.    # (1, height, width, channels) + normalize

    #Update the GUI
    state.online_image_predicted_label  = class_names[np.argmax(state.model.predict(image))]
    state.online_image_count += 1

In [ ]:
Gui(page=p1+p2).run(port=port, dark_mode=False)

port += 1

1/1 [==============================] - 0s 17ms/step
